**Install Packages**

In [ ]:
!pip install pycaret
!pip install mysql-connector-python
!pip install sqlalchemy==1.4.32
!pip install meteostat

**Import Packages**

In [ ]:
import pycaret
import pandas as pd
import numpy as np
import sqlalchemy
import mysql.connector

from pycaret import classification
from google.colab import drive
from datetime import datetime
from google.colab.data_table import DataTable
from google.colab import files
drive.mount('/content/drive')
DataTable.max_columns = 100

Player_Positions = pd.read_csv("/content/drive/MyDrive/MLB_Player_Positions.csv")

Mounted at /content/drive


**Load Data**

In [ ]:
engine = sqlalchemy.create_engine()
Hit_Data_Original = pd.read_sql(sql = "SELECT * FROM baseball_historical_matchup", con = engine).dropna()
print(len(Hit_Data_Original))

9503


**Clean Data**

In [ ]:
Features = ['pitching_gamesPlayed', 'pitching_gamesStarted', 'pitching_groundOuts',
       'pitching_airOuts', 'pitching_runs', 'pitching_doubles',
       'pitching_triples', 'pitching_homeRuns', 'pitching_strikeOuts',
       'pitching_baseOnBalls', 'pitching_intentionalWalks', 'pitching_hits',
       'pitching_hitByPitch', 'pitching_avg', 'pitching_atBats',
       'pitching_obp', 'pitching_slg', 'pitching_ops',
       'pitching_caughtStealing', 'pitching_stolenBases', 'pitching_groundIntoDoublePlay',
       'pitching_numberOfPitches',
       'pitching_wins', 'pitching_losses', 'pitching_saves',
       'pitching_saveOpportunities', 'pitching_holds', 'pitching_blownSaves',
       'pitching_earnedRuns', 'pitching_battersFaced',
       'pitching_outs', 'pitching_gamesPitched', 'pitching_completeGames',
       'pitching_shutouts', 'pitching_strikes', 'pitching_strikePercentage',
       'pitching_hitBatsmen', 'pitching_balks', 'pitching_wildPitches',
       'pitching_pickoffs', 'pitching_totalBases','pitching_gamesFinished',
       'pitching_inheritedRunners', 'pitching_inheritedRunnersScored',
       'pitching_catchersInterference', 'pitching_sacBunts',
       'pitching_sacFlies', 'pitching_name', 'pitching_id',
       'batting_gamesPlayed', 'batting_groundOuts', 'batting_airOuts',
       'batting_runs', 'batting_doubles', 'batting_triples',
       'batting_homeRuns', 'batting_strikeOuts', 'batting_baseOnBalls',
       'batting_intentionalWalks', 'batting_hits', 'batting_hitByPitch',
       'batting_avg', 'batting_atBats', 'batting_obp', 'batting_slg',
       'batting_ops', 'batting_caughtStealing', 'batting_stolenBases', 'batting_groundIntoDoublePlay',
       'batting_numberOfPitches', 'batting_plateAppearances',
       'batting_totalBases', 'batting_rbi', 'batting_leftOnBase',
       'batting_sacBunts', 'batting_sacFlies', 'batting_catchersInterference',
        'batting_name', 'batting_id',
       'batting_hit_recorded', 'game_id', 'game_venue', 'game_date']

#Data[["batting_avg","batting_obp","batting_slg","batting_ops","pitching_strikePercentage", "pitching_avg" , "pitching_era", "pitching_obp", "pitching_slg", "pitching_ops"]] = Data[["batting_avg","batting_obp","batting_slg","batting_ops","pitching_strikePercentage", "pitching_avg" , "pitching_era", "pitching_obp", "pitching_slg", "pitching_ops"]].replace([".---", "-.--"], np.nan).astype(float)

Hit_Real_Features = ["pitching_gamesPlayed","pitching_runs", "pitching_strikeOuts", "pitching_hits","pitching_id","pitching_name", "batting_gamesPlayed", "batting_runs","batting_strikeOuts", "batting_hits","batting_rbi", "batting_hit_recorded","batting_id","batting_name","temp" ,"game_id", "game_venue","game_date"]       
Hit_Data = Hit_Data_Original[Hit_Real_Features]
Hit_Data.info()

**Model Setup**

In [ ]:
Hit_Classification_Setup = pycaret.classification.setup(data = Hit_Data, target = "batting_hit_recorded", train_size = .70, use_gpu = True, categorical_features = ["pitching_name","batting_name", "game_venue"], ignore_features = ["pitching_id","batting_id","game_id","game_date"])


,Description,Value
0,Session id,6601
1,Target,batting_hit_recorded
2,Target type,Binary
3,Original data shape,"(9503, 18)"
4,Transformed data shape,"(9503, 14)"
5,Transformed train set shape,"(6652, 14)"
6,Transformed test set shape,"(2851, 14)"
7,Ignore features,4
8,Numeric features,10
9,Categorical features,3


**Build/Train Model**

In [ ]:
Hit_Logistic_Regression =  Hit_Classification_Setup.create_model("lr", probability_threshold = 0.50)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6006,0.5950,0.8519,0.6108,0.7115,0.1167,0.1352
1,0.5916,0.5705,0.8468,0.6048,0.7056,0.0958,0.1116
2,0.6256,0.6277,0.8571,0.6298,0.7261,0.1762,0.1984
3,0.6060,0.5911,0.8779,0.6112,0.7207,0.1199,0.1452
4,0.6195,0.5983,0.8753,0.6218,0.7271,0.1550,0.1821
5,0.6075,0.5942,0.8442,0.6179,0.7135,0.1356,0.1534
6,0.6180,0.6168,0.8623,0.6229,0.7233,0.1557,0.1788
7,0.6195,0.6318,0.8831,0.6204,0.7288,0.1524,0.1819
8,0.6286,0.6299,0.8828,0.6266,0.7330,0.1772,0.2079


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

**Finalize & Save Model**

In [ ]:
Finalized_Hit_Model = pycaret.classification.finalize_model(Hit_Logistic_Regression)
hit_save_to_file_string = f"{datetime.today().strftime('%Y-%m-%d')} Batter Hit LR Model"
pycaret.classification.save_model(Finalized_Hit_Model, hit_save_to_file_string)
files.download(f"{hit_save_to_file_string}.pkl")

Transformation Pipeline and Model Successfully Saved


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>